In [1]:
import numpy as np

# The binary Golay code G_24

In [3]:
# the (extended) binary Golay code G24

## all the 2^12 vectors of (Z_2)^12
C = np.array([[0,0,0,0,0,0, 0,0,0,0,0,0]])
unit = np.array([1,0,0,0,0,0, 0,0,0,0,0,0])
for i in range(12):
    C = np.concatenate([C, C + unit])
    unit = np.roll(unit, 1)
### Sanity Check
print("len(C) == 2**12 ?:", len(C) == 2**12)

## Generators cited from [Conway-Sloane Ch.10 Sec.2.1. Proof of Thm.7 p.275]
## Conway-Sloane->Python: 0->0, 1->1, ..., 22->22, inf->23.
T = 1
F = 0
GOLAY_GENERATORS = np.array([
    [T,T,T,T,T, F,F,T,F,F, T,F,T,F,F, F,F,F,F,F, F,F,F,F],
    [F, T,T,T,T,T, F,F,T,F,F, T,F,T,F,F, F,F,F,F,F, F,F,F],
    [F,F, T,T,T,T,T, F,F,T,F,F, T,F,T,F,F, F,F,F,F,F, F,F],
    [F,F,F, T,T,T,T,T, F,F,T,F,F, T,F,T,F,F, F,F,F,F,F, F],
    [F,F,F,F, T,T,T,T,T, F,F,T,F,F, T,F,T,F,F, F,F,F,F,F],
    [F, F,F,F,F, T,T,T,T,T, F,F,T,F,F, T,F,T,F,F, F,F,F,F],
    [F,F, F,F,F,F, T,T,T,T,T, F,F,T,F,F, T,F,T,F,F, F,F,F],
    [F,F,F, F,F,F,F, T,T,T,T,T, F,F,T,F,F, T,F,T,F,F, F,F],
    [F,F,F,F, F,F,F,F, T,T,T,T,T, F,F,T,F,F, T,F,T,F,F, F],
    [F,F,F,F,F, F,F,F,F, T,T,T,T,T, F,F,T,F,F, T,F,T,F,F],
    [F, F,F,F,F,F, F,F,F,F, T,T,T,T,T, F,F,T,F,F, T,F,T,F],
    
    [T,T,T,T,T, T,T,T,T,T, T,T,T,T,T, T,T,T,T,T, T,T,T,T]
])

## Calculate G_24
GOLAY = C @ GOLAY_GENERATORS % 2

### Sanity Check
## octads
GOLAY_octads = [w for w in GOLAY if np.count_nonzero(w) == 8]
print("Are there 759 octads?:", len(GOLAY_octads) == 759)
## dodecads
GOLAY_dodecads = [w for w in GOLAY if np.count_nonzero(w) == 12]
print("Are there 2576 dodecads?:", len(GOLAY_dodecads) == 2576)

len(C) == 2**12 ?: True
Are there 759 octads?: True
Are there 2576 dodecads?: True


# Function for finding orthogonal dodecads

In [5]:
# M: symmetric matrix
# d <= len(M)
# Find d indices ans such that M restrected to these indices is a diagonal matrix.
# (In particular, if the diagonal elements of M are non-zero, then M restrected to these indeces is an identity matrix under the rescaled basis.)

def find_diag_submat(M, d):
    n = len(M)
    index_of_zero = [[i for i in range(n)]]
    ans = []
    depth = 0
    
    # Algorithm
    # 1. Set i = index_of_zero[-1][0] and append i to ans.
    # 2. Search the i-th row of M but only M[i][j] for j in index_of_zero[-1][1:], and collect the indices j such that M[i][j] == 0.
    # 3. The array of such j's is appended to index_of_zero.
    # 4. If len(ans) is still less than d, then go back to 1.
    #    Here, if index_of_zero[-1] is empty and we cannot continue 1., then ans[-1] turns out to be inappropriate for ans. 
    #    So pop ans[-1] from ans, pop the empty array from index_of_zero, and pop index_of_zero[-1][0] from index_of_zero[-1].
    while depth < d:
        if index_of_zero[-1] == []:
            if depth == 0:
                return False
            else:
                index_of_zero.pop(-1)
                index_of_zero[-1].pop(0)
                ans.pop(-1)
                depth -= 1
                continue
        ans.append(index_of_zero[-1][0])
        depth += 1
        index_of_zero.append([i for i in index_of_zero[-1][1:] if M[index_of_zero[-1][0]][i] == 0])
    return ans

In [7]:
###Sanity Check
A = [
    [1,0,3,4,0],
    [0,1,8,9,0],
    [3,8,1,5,6],
    [4,9,5,1,2],
    [0,0,6,2,1]
]
print(find_diag_submat(A, 2)) # [0, 1]
print(find_diag_submat(A, 3)) # [0, 1, 4]
print(find_diag_submat(A, 4)) # False

[0, 1]
[0, 1, 4]
False


 # Find orthogonal dodecads

In [9]:
# Find 23 dodecads with the first entry 0, such that any two of the 24 vectors, (1,...,1) and (-1)^codeword for those dodecads, are orthogonal.
# Then the matix consisting of these 24 vectors is the matrix form of the lattice isometry (up to multiplication of 1/(2*sqrt(6)))
# from the odd Leech lattice constructed from a ternary code to that from the binary Golay code.

## dodecads with the first entry 0
GOLAY_dodecads_0 = [w for w in GOLAY_dodecads if w[0] == 0]
## (-1)^dodecads
GOLAY_dodecads_0_pm = np.array([[1-2*x for x in dodecad] for dodecad in GOLAY_dodecads_0])
## Find the 23 dodecads.
index_of_ortho_dodecads = find_diag_submat(GOLAY_dodecads_0_pm @ GOLAY_dodecads_0_pm.T, 23)

## Make the matrix form of the lattice isometry.
ortho_mat_from_dodecads = [[1 for i in range(24)]] # the first row is [1,...,1]
for i in index_of_ortho_dodecads:
    ortho_mat_from_dodecads.append(list(GOLAY_dodecads_0_pm[i]))
print(np.matrix(ortho_mat_from_dodecads))

[[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1]
 [ 1 -1  1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1  1 -1 -1  1  1  1  1  1  1  1  1]
 [ 1  1 -1 -1 -1  1 -1  1  1 -1 -1  1  1 -1 -1 -1 -1 -1 -1  1  1  1  1  1]
 [ 1 -1 -1  1  1 -1 -1 -1 -1  1  1 -1 -1 -1  1  1 -1 -1 -1  1  1  1  1  1]
 [ 1 -1 -1 -1 -1 -1  1  1  1  1  1 -1  1  1  1 -1 -1  1 -1 -1 -1 -1  1  1]
 [ 1  1 -1  1  1  1  1 -1 -1 -1 -1  1 -1  1 -1  1 -1  1 -1 -1 -1 -1  1  1]
 [ 1  1 -1 -1 -1 -1 -1 -1 -1  1  1  1  1 -1 -1  1  1  1  1 -1 -1  1 -1  1]
 [ 1 -1 -1  1  1  1 -1  1  1 -1 -1 -1 -1 -1  1 -1  1  1  1 -1 -1  1 -1  1]
 [ 1 -1  1  1  1 -1  1  1 -1  1 -1  1  1 -1 -1 -1 -1 -1  1  1 -1 -1 -1  1]
 [ 1  1  1 -1 -1  1  1 -1  1 -1  1 -1 -1 -1  1  1 -1 -1  1  1 -1 -1 -1  1]
 [ 1  1  1  1 -1 -1 -1 -1  1  1 -1  1 -1  1  1 -1  1 -1 -1 -1  1 -1 -1  1]
 [ 1 -1  1 -1  1  1 -1  1 -1 -1  1 -1  1  1 -1  1  1 -1 -1 -1  1 -1 -1  1]
 [ 1 -1 -1 -1 -1  1  1  1 -1  1 -1  1 -1 -1  1  1  1  1 -1  1  1 -1 -1 -1]
 [ 1  1 -1  1  1 -1  1 -1

In [11]:
### Sanity Check
print(np.matrix(ortho_mat_from_dodecads) @ np.matrix(ortho_mat_from_dodecads).T / 24) # must be the identity matrix.

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.

# Check 1: cocycle factors

## The cocycle factor \varepsilon(k, k')

### the basis of 2sqrt(2) * (odd Leech lattice O_24 from G_24)

In [13]:
# odd Leech lattice O_24 from G_24
# O_24 = ((1/sqrt(2))G_24 + sqrt(2)Z^24_+) \cup ((1/2sqrt(2))(1, ... ,1) + (1/sqrt(2))G_24 + sqrt(2)Z^24_+)

# the basis of 2sqrt(2) * O_24
T = 2
F = 0
ODDLEECH_BASIS = np.array([
    [1,1,1,1, 1,1,1,1, 1,1,1,1, 1,1,1,1, 1,1,1,1, 1,1,1,1], # 2sqrt(2)*e0; e0 @ e0 = odd
    
    [F, F,F,F,F,F, F,F,F,F, T,T,T,T,T, F,F,T,F,F, T,F,T,F], # 2sqrt(2)*e1
    [F,F,F,F,F, F,F,F,F, T,T,T,T,T, F,F,T,F,F, T,F,T,F,F], # 2sqrt(2)*e2
    [F,F,F,F, F,F,F,F, T,T,T,T,T, F,F,T,F,F, T,F,T,F,F, F], # 2sqrt(2)*e3
    [F,F,F, F,F,F,F, T,T,T,T,T, F,F,T,F,F, T,F,T,F,F, F,F],
    [F,F, F,F,F,F, T,T,T,T,T, F,F,T,F,F, T,F,T,F,F, F,F,F],
    [F, F,F,F,F, T,T,T,T,T, F,F,T,F,F, T,F,T,F,F, F,F,F,F],
    [F,F,F,F, T,T,T,T,T, F,F,T,F,F, T,F,T,F,F, F,F,F,F,F],
    [F,F,F, T,T,T,T,T, F,F,T,F,F, T,F,T,F,F, F,F,F,F,F, F],
    [F,F, T,T,T,T,T, F,F,T,F,F, T,F,T,F,F, F,F,F,F,F, F,F],
    [F, T,T,T,T,T, F,F,T,F,F, T,F,T,F,F, F,F,F,F,F, F,F,F],
    [T,T,T,T,T, F,F,T,F,F, T,F,T,F,F, F,F,F,F,F, F,F,F,F], # 2sqrt(2)*e11

    [4,0,0,0, 0,0,0,0, 0,0,0,4, 0,0,0,0, 0,0,0,0, 0,0,0,0], # 2sqrt(2)*e12
    [4,0,0,0, 0,0,0,0, 0,0,4,0, 0,0,0,0, 0,0,0,0, 0,0,0,0],
    [4,0,0,0, 0,0,0,0, 0,4,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0],
    [4,0,0,0, 0,0,0,0, 4,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0],
    [4,0,0,0, 0,0,0,4, 0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0],
    [4,0,0,0, 0,0,4,0, 0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0],
    [4,0,0,0, 0,4,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0],
    [4,0,0,0, 4,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0],
    [4,0,0,4, 0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0],
    [4,0,4,0, 0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0],
    [4,4,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0],
    [8,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0] # 2sqrt(2)*e23
])

for b in ODDLEECH_BASIS:
    print(b)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[0 0 0 0 0 0 0 0 0 0 2 2 2 2 2 0 0 2 0 0 2 0 2 0]
[0 0 0 0 0 0 0 0 0 2 2 2 2 2 0 0 2 0 0 2 0 2 0 0]
[0 0 0 0 0 0 0 0 2 2 2 2 2 0 0 2 0 0 2 0 2 0 0 0]
[0 0 0 0 0 0 0 2 2 2 2 2 0 0 2 0 0 2 0 2 0 0 0 0]
[0 0 0 0 0 0 2 2 2 2 2 0 0 2 0 0 2 0 2 0 0 0 0 0]
[0 0 0 0 0 2 2 2 2 2 0 0 2 0 0 2 0 2 0 0 0 0 0 0]
[0 0 0 0 2 2 2 2 2 0 0 2 0 0 2 0 2 0 0 0 0 0 0 0]
[0 0 0 2 2 2 2 2 0 0 2 0 0 2 0 2 0 0 0 0 0 0 0 0]
[0 0 2 2 2 2 2 0 0 2 0 0 2 0 2 0 0 0 0 0 0 0 0 0]
[0 2 2 2 2 2 0 0 2 0 0 2 0 2 0 0 0 0 0 0 0 0 0 0]
[2 2 2 2 2 0 0 2 0 0 2 0 2 0 0 0 0 0 0 0 0 0 0 0]
[4 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0]
[4 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0]
[4 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[4 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[4 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[4 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[4 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[4 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


### fuction to calculate the components of a vector in O_24 with respect to the basis

In [15]:
# return the component [c_0, ..., c_23] of v = 2sqrt(2) * (lattice point in O_24) = 2sqrt(2) * (c_0e_0+...+c_23e_23)
def component(v):
    copy_of_v = np.array([v[i] for i in range(24)])
    ans = [0 for i in range(24)]
    for i in range(24):
        ans[i] = copy_of_v[23-i] / ODDLEECH_BASIS[i][23-i]
        copy_of_v = copy_of_v - ans[i] * ODDLEECH_BASIS[i]
    if np.count_nonzero(copy_of_v) != 0:
        print("fail to decompose!")
        return copy_of_v
    else:
        return ans
    
# return 2sqrt(2) * (c_0e_0+...+c_23e_23) for c = [c_0, ... ,c_23]
def comp_to_vec(c):
    ans = [0 for i in range(24)]
    for i in range(24):
        ans = ans + c[i] * ODDLEECH_BASIS[i]
    return ans
    
### Sanity Check
print(component([4,-4,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0]))
print(comp_to_vec(component([4,-4,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0])))

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 1.0]
[ 4. -4.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.]


### the intersection form of O_24

In [17]:
# the intersection form of the basis of O_24
INTERSECTION_FORM = ODDLEECH_BASIS @ ODDLEECH_BASIS.T / 8
print(INTERSECTION_FORM)

[[3. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [2. 4. 2. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [2. 2. 4. 2. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [2. 2. 2. 4. 2. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [2. 2. 2. 2. 4. 2. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
 [2. 1. 2. 2. 2. 4. 2. 2. 2. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
 [2. 1. 1. 2. 2. 2. 4. 2. 2. 2. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [2. 1. 1. 1. 2. 2. 2. 4. 2. 2. 2. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
 [2. 1. 1. 1. 1. 2. 2. 2. 4. 2. 2. 2. 0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0.]
 [2. 1. 1. 1. 1. 1. 2. 2. 2. 4. 2. 2. 0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0.]
 [2. 1. 1. 1. 1. 1. 1. 2. 2. 2. 4. 2. 1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 0.]
 [2. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 4. 1. 2. 1. 1. 2. 1. 1. 2. 2. 2. 2. 2.]
 [1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1. 1. 4. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 4.]
 [1. 1. 1. 1. 1. 1. 0. 0.

### function to calculate the cocycle factor \varepsilon(k, k') of k, k' in O_24

In [19]:
# return k_1 * k_2 (mod 2) for k_i = v_i/2sqrt(2) in O_24, v_i in 2sqrt(2)*O_24
def ast_prod(v1, v2):
    v1_comp = component(v1)
    v2_comp = component(v2)
    ans = 0
    for i in range(24):
        for j in range(i):
            ans = ans + v1_comp[i] * v2_comp[j] * INTERSECTION_FORM[i][j]
    return ans

# return the cocycle factor \varepsilon(k_1, k_2) for k_i = v_i/2sqrt(2) in O_24, v_i in 2sqrt(2)*O_24
def cocycle_factor(v1, v2):
    return (-1) ** ast_prod(v1, v2)

### Sanity Check
print(ast_prod(ODDLEECH_BASIS[5], 3 * ODDLEECH_BASIS[1]))
print(cocycle_factor(ODDLEECH_BASIS[5], 3 * ODDLEECH_BASIS[1]))

3.0
-1.0


## Check \varepsilon(k, -k) = 1 for k = (-1)^w / 2sqrt(2)

In [21]:
# for v = (-1)^w in ortho_mat_from_dodecads, check \varepsilon(k, -k) = 1 where k = v / 2sqrt(2).

for v in ortho_mat_from_dodecads:
    if cocycle_factor(v, [-x for x in v]) != 1:
        print("failed!")
print("completed")

completed


# Check 2: J(z) is in the direction of (1,0,...,0)

In [23]:
# J(z) is \sum_{(-1)^w in ortho_mat_from_dodecads} (-1)^w \cdot \partial X(z) (up to constant multiplication).

print(sum(np.array(ortho_mat_from_dodecads)))

[24  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


# Check 3: T(z) is \sum_i (\partial X^i(z))^2 (up to constant multiplication)

In [25]:
# T(z) is \sum_{(-1)^w in ortho_mat_from_dodecads} ( (-1)^w \cdot \partial X(z) )^2 (up to constant multiplication).
# It suffices to check that the cross terms \parital X^i(z) \partial X^j(z) vanish.

for i in range(24):
    for j in range(i+1, 24):
        cross_term_ij = 0
        for v in ortho_mat_from_dodecads:
            cross_term_ij = cross_term_ij + v[i] * v[j]
        if cross_term_ij != 0:
            print("failed!")
print("completed")

completed
